In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),
    ]
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example에 사용된 변수와 example_template에 사용되는 변수의 이름이 반드시 같아야 함
example_template = [
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
]

# ChatPromptTemplate을 이용하여 형식을 지정
example_prompt = ChatPromptTemplate.from_messages(example_template)

# 아래처럼 하나로 만들어 줄 수도 있음
# example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# 랭체인이 알아서 각각의 예제 리스트들을 사용하여 prompt를 형식화함
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    example_prompt,
    ("human", "What do you know about {country}?")
])

# 위에까지가 언어 모델(LM)이 완성된 것임

chain = final_prompt | chat

chain.invoke({
    "country": "Germany"
})



        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ')